In [1]:
from pathlib import Path

import torch
torch.set_default_tensor_type(torch.cuda.FloatTensor)
torch.set_grad_enabled(False)

from tqdm.auto import tqdm

from lib.bayesn import get_cpdataset, get_ranges, load


N = 500
DATA_GEN = f'{N}-cosmo-ir'
GUIDE_KEY = 'hpmvn'
CONDITION_KEY = ''
SUFFIX = ''  # '-negrenyi'

EXPERIMENT_NAME = '-'.join((_ for _ in (GUIDE_KEY, CONDITION_KEY) if _))
SAVENAME = f'{EXPERIMENT_NAME}{SUFFIX}'
RESDIR = Path(f'res-{DATA_GEN}')

NNTRAIN = RESDIR / 'nntrain' / f'{SAVENAME}-nntrain-{{i}}.pt'


guidename = RESDIR / f'{SAVENAME}-guide.pt'

config = load(EXPERIMENT_NAME, DATA_GEN, N, guidename=guidename)
config.guide.eval()

Guide(
  (hpmvn): HierarchicPartialMultivariateNormalSamplingGroup(8 sites, torch.Size([1505]))
)

In [2]:
ranges = get_ranges(config, ignore=('Om0', 'Ode0', 'delta_M'), nsigma=3)
dataset = get_cpdataset(config, CONDITION_KEY, ranges, (0.1, 0.5, 0.5, 0.9))

In [ ]:
from more_itertools import take

for i in range(7):
    torch.save(
        take(N := 3000, tqdm(dataset, total=N)), NNTRAIN.format(i=i)
    )